In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv('edu_enrollees.csv')
df.head(3)

,enrollee_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target,Xgrp
0,8949.0,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36.0,1.0,train
1,29725.0,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47.0,0.0,train
2,11561.0,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83.0,0.0,train


In [13]:
df.shape

(19158, 15)

In [3]:
df.columns

Index(['enrollee_id', 'city', 'city_development_index', 'gender',
       'relevant_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'target', 'Xgrp'],
      dtype='object')

In [4]:
len(df.columns)

15

In [8]:
df_base = df.drop(columns = ['city','company_size','company_type'])
df_base.columns

Index(['enrollee_id', 'city_development_index', 'gender',
       'relevant_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'last_new_job', 'training_hours',
       'target', 'Xgrp'],
      dtype='object')

In [10]:
df_base.dtypes

enrollee_id               float64
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
last_new_job               object
training_hours            float64
target                    float64
Xgrp                       object
dtype: object

In [11]:
df_base.head(3)

,enrollee_id,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,last_new_job,training_hours,target,Xgrp
0,8949.0,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1,36.0,1.0,train
1,29725.0,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,>4,47.0,0.0,train
2,11561.0,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,never,83.0,0.0,train


In [12]:
df.isnull().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    4508
relevant_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
company_size              5938
company_type              6140
last_new_job               423
training_hours               0
target                       0
Xgrp                         0
dtype: int64

In [17]:
len(df_base.dropna(axis=0))

12477

In [18]:
df_base = df_base.dropna(axis = 0)

In [30]:
df_base.head(3)

,enrollee_id,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,last_new_job,training_hours,target,Xgrp
0,8949.0,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1,36.0,1.0,train
1,29725.0,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,>4,47.0,0.0,train
4,666.0,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,4,8.0,0.0,train


In [35]:
df_base = df_base[df_base['experience'] != '>20']
len(df_base)

9842

In [36]:
df_base = df_base[df_base['experience'] != '<1']
len(df_base)

9653

In [37]:
df_base['experience'] = df_base['experience'].astype(int)

In [38]:
df_base = df_base[df_base['last_new_job'] != '>4']
len(df_base)

8340

In [39]:
df_base = df_base[df_base['last_new_job'] != 'never']
len(df_base)

7522

In [40]:
df_base['last_new_job'] = df_base['last_new_job'].astype(int)

In [41]:
df_base.dtypes

enrollee_id               float64
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                  int32
last_new_job                int32
training_hours            float64
target                    float64
Xgrp                       object
dtype: object

In [42]:
df_base.shape

(7522, 12)

# Q1

In [43]:
df_base.head(3)

,enrollee_id,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,last_new_job,training_hours,target,Xgrp
8,27107.0,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,7,1,46.0,1.0,train
11,23853.0,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,5,1,108.0,0.0,train
19,11399.0,0.827,Female,Has relevant experience,no_enrollment,Graduate,Arts,4,1,132.0,1.0,train


In [44]:
df_base.columns

Index(['enrollee_id', 'city_development_index', 'gender',
       'relevant_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'last_new_job', 'training_hours',
       'target', 'Xgrp'],
      dtype='object')

In [51]:
val_exp_ok = df_base[df_base['relevant_experience'] == 'Has relevant experience']
val_exp_no = df_base[df_base['relevant_experience'] == 'No relevant experience']
len(val_exp_ok), len(val_exp_no)

(6109, 1413)

In [58]:
df_base['target'].unique()

array([1., 0.])

In [60]:
val_exp_ok_target = val_exp_ok[val_exp_ok['target'] == 1.0]
val_exp_no_target = val_exp_ok[val_exp_ok['target'] == 0.0]
len(val_exp_ok_target), len(val_exp_no_target)

(1319, 4790)

In [67]:
result_q1 = ((len(val_exp_no_target)/len(val_exp_no))/(len(val_exp_ok_target)/len(val_exp_ok)))
result_q1, round(result_q1, 2)

(15.700687915258884, 15.7)

# Q2

In [68]:
df_base.head(3)

,enrollee_id,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,last_new_job,training_hours,target,Xgrp
8,27107.0,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,7,1,46.0,1.0,train
11,23853.0,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,5,1,108.0,0.0,train
19,11399.0,0.827,Female,Has relevant experience,no_enrollment,Graduate,Arts,4,1,132.0,1.0,train


In [71]:
df_base.columns

Index(['enrollee_id', 'city_development_index', 'gender',
       'relevant_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'last_new_job', 'training_hours',
       'target', 'Xgrp'],
      dtype='object')

In [72]:
df_base.dtypes

enrollee_id               float64
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                  int32
last_new_job                int32
training_hours            float64
target                    float64
Xgrp                       object
dtype: object

# ========================================

In [73]:
df = df.drop(columns = ['city', 'company_size', 'company_type'])

In [74]:
df.dtypes

enrollee_id               float64
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
last_new_job               object
training_hours            float64
target                    float64
Xgrp                       object
dtype: object

In [75]:
df.isna().sum()

enrollee_id                  0
city_development_index       0
gender                    4508
relevant_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
last_new_job               423
training_hours               0
target                       0
Xgrp                         0
dtype: int64

In [76]:
df = df.dropna()

In [78]:
df.isna().sum()

enrollee_id               0
city_development_index    0
gender                    0
relevant_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
last_new_job              0
training_hours            0
target                    0
Xgrp                      0
dtype: int64

In [79]:
df['experience'].unique(), df['last_new_job'].unique()

(array(['>20', '15', '13', '7', '5', '16', '4', '11', '<1', '18', '19',
        '12', '10', '9', '2', '6', '14', '3', '8', '20', '17', '1'],
       dtype=object),
 array(['1', '>4', '4', '3', '2', 'never'], dtype=object))

In [83]:
#df = df.loc[df['experience'] != ">20", ]
df = df.loc[~df['experience'].isin(['>20', '<1']),]
df = df.loc[~df['last_new_job'].isin(['>4', 'never']), ]

In [84]:
df['experience'] = df['experience'].astype('int')
df['last_new_job'] = df['last_new_job'].astype('int')

In [85]:
len(df)

7522

In [86]:
df_base = df.reset_index(drop = True)

# Q1

In [87]:
df_q1 = df_base[['relevant_experience', 'target']].copy()
df_q1.head(3)

,relevant_experience,target
0,Has relevant experience,1.0
1,Has relevant experience,0.0
2,Has relevant experience,1.0


In [91]:
df_q1_subA = df_q1.loc[df_q1['relevant_experience'] == 'No relevant experience',  ]
df_q1_subB = df_q1.loc[df_q1['relevant_experience'] == 'Has relevant experience', ]

In [94]:
df_q1_subA['target'].value_counts(normalize = True)[1]

0.38287331917905165

In [95]:
df_q1_subB['target'].value_counts(normalize = True)[1]

0.21591095105581928

In [96]:
stat_A = df_q1_subA['target'].value_counts(normalize = True)[1]
stat_B = df_q1_subB['target'].value_counts(normalize = True)[1]
round(stat_A / stat_B, 2)

1.77

In [98]:
df_q1.groupby('relevant_experience')['target'].mean()

relevant_experience
Has relevant experience    0.215911
No relevant experience     0.382873
Name: target, dtype: float64

# Q2

In [99]:
df_base.columns

Index(['enrollee_id', 'city_development_index', 'gender',
       'relevant_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'last_new_job', 'training_hours',
       'target', 'Xgrp'],
      dtype='object')

In [100]:
df_q2_cat = df_base.loc[:, 'gender':'major_discipline']
df_q2_cat.head(2)

,gender,relevant_experience,enrolled_university,education_level,major_discipline
0,Male,Has relevant experience,no_enrollment,Graduate,STEM
1,Male,Has relevant experience,no_enrollment,Graduate,STEM


In [101]:
df_q2_dum = pd.get_dummies(df_q2_cat, columns = df_q2_cat.columns)
df_q2_dum.head(2)

,gender_Female,gender_Male,gender_Other,relevant_experience_Has relevant experience,relevant_experience_No relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,enrolled_university_no_enrollment,education_level_Graduate,education_level_Masters,education_level_Phd,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other,major_discipline_STEM
0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1
1,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1


In [ ]:
drop_cols = 
df_q2_dum2 = df_q2_dum.drop(columns = drop_cols)
df_q2_dum2.head(2)

In [102]:
df_q2_dum.columns.to_series().reset_index(drop = True).reset_index()

,index,0
0,0,gender_Female
1,1,gender_Male
2,2,gender_Other
3,3,relevant_experience_Has relevant experience
4,4,relevant_experience_No relevant experience
5,5,enrolled_university_Full time course
6,6,enrolled_university_Part time course
7,7,enrolled_university_no_enrollment
8,8,education_level_Graduate
9,9,education_level_Masters
